In [32]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import pathlib
import random

from credentials import SAMPLE_DB, FULL_DB
from librarian import SAMPLE_DATA_DIR

import encoders
import db_connect
import helpers

## Config
This notebook has a lot of options to adjust, most of which are controlled here.

In [33]:
DATA_DIR = SAMPLE_DATA_DIR
CHUNK_SIZE = 512
PROCESSING_UNITS = CHUNK_SIZE // 4
LAYER_UNITS = max(1, CHUNK_SIZE // 10)

INFER_TEXT = True
INFER_KEY = not INFER_TEXT

USE_CUSTOM_LOSS = True
USE_CUSTOM_OUTPUT_ACTIVATION = True

OUTPUT_MIN = 0
OUTPUT_MAX = len(encoders.CHARSET)-1
CUSTOM_LOSS_MODULO = OUTPUT_MAX+1

if INFER_TEXT:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mean_squared_error"
    OUTPUT_SIZE = CHUNK_SIZE
    OPTIMIZER = "sgd"
else:
    MAIN_ACCURACY_METRIC = "mae"
    LOSS_METRIC = "mae"
    OUTPUT_SIZE = 1 # actually depends on cipher
    OPTIMIZER = "adamax"    

#ENCRYPTED_FILE_LIMIT = 10 # -1 to disable limit
ENCRYPTED_FILE_LIMIT = -1 # -1 to disable limit

BASE_TRAIN_PCT = 0.75   # Start here. If train or test count would exceed the max, reduce it. Note 0.75 is the default.
MAX_TRAIN_COUNT = 10000 # -1 to disable
MAX_TEST_COUNT = 10000 # -1 to disable
SPLIT_SEED = 42

LOAD_BEST_MODEL = False # If False, a new model will be created from scratch
SAVE_BEST_MODEL = True
BEST_PATH = './best.keras'

# Whether to run the tuner or the hard-coded network build code
TUNE_NETWORK = False
BUILD_NETWORK = not TUNE_NETWORK

TRAIN_MODEL = BUILD_NETWORK

EPOCHS = 5
BATCH_SIZE = 32 # Default is 32

# Data Retrieval and Structuring

In [34]:
# Get database IDs for encoders and key types

encoder_ids= {}
key_type_ids = {}

db = db_connect.DB(SAMPLE_DB)
with db.get_session() as session:
    for encoder in encoders.ALL_ENCODER_NAMES:
        id = db.get_encoder_id(session, encoder)
        encoder_ids[encoder] = id

    print(f"Encoder IDs: {encoder_ids}")

    for key_type in encoders.KEY_NAMES:
        id = db.get_key_type_id(session, key_type)
        key_type_ids[key_type] = id

    print(f"Key Type IDs: {key_type_ids}")

Encoder IDs: {'None': 1, 'Simplifier': 2, 'Caesar Cipher': 3, 'Substitution Cipher': 4, 'Enigma Machine': 5}
Key Type IDs: {'Character Offset': 1, 'Character Map': 2, 'Rotor Settings': 3}


In [35]:
# Map source ID to plaintext file (1) details, and source ID to corresponding ciphertext files (1+) details
sid_to_p = {}
sid_to_c = {}

cipher_id = encoder_ids[encoders.ENCODER_CAESAR]
with db.get_session() as session:
    # Get all files encrypted with the cipher we care about
    encrypted_files = db.get_files_by_source_and_encoder(session, -1, cipher_id)

    if len(encrypted_files) > ENCRYPTED_FILE_LIMIT and ENCRYPTED_FILE_LIMIT > 0:
        encrypted_files = random.sample(encrypted_files, ENCRYPTED_FILE_LIMIT)

    for c in encrypted_files:
        sid = c.source_id
    
        if sid not in sid_to_p:
            plaintext_ids = db.get_files_by_source_and_encoder(session, sid, encoder_ids[encoders.ENCODER_SIMPLIFIER])
            if len(plaintext_ids) != 1:
                raise Exception(f"Found {len(plaintext_ids)} plaintexts for source ID {sid}; should be exactly 1")
            sid_to_p[sid] = plaintext_ids[0]

        if sid not in sid_to_c:
            sid_to_c[sid] = []
        sid_to_c[sid].append(c)

len(sid_to_p), len(sid_to_c)

(6, 6)

In [36]:
# Build up the features (X, the cipher texts as offsets) and targets (y, either the plain texts as offsets OR the key).
# Note targets are not necessarily unique.
X = []
y = []

with db.get_session() as session:
    for sid in sid_to_p:
        if INFER_TEXT:
            plaintext = encoders.string_to_offsets(helpers.read_text_file(sid_to_p[sid].path))
            target_chunks = helpers.chunkify(plaintext, CHUNK_SIZE)    
    
        for c in sid_to_c[sid]:
            ciphertext = encoders.string_to_offsets(helpers.read_text_file(c.path))
            feature_chunks = helpers.chunkify(ciphertext, CHUNK_SIZE)

            if INFER_KEY:
                key_value = float(db.get_key_by_id(session, c.key_id).value)
    
            for i in range (len(feature_chunks)):
                X.append(np.array(feature_chunks[i]).astype(float))

                if INFER_TEXT:
                    y.append(np.array(target_chunks[i]).astype(float))

                if INFER_KEY:
                    y.append(key_value)

X = np.array(X)
y = np.array(y)

X.shape, y.shape, X[0].shape, y[0].shape, X[0], y[0]

((193280, 512),
 (193280, 512),
 (512,),
 (512,),
 array([46., 46.,  0.,  2., 61., 50.,  5., 49., 51., 50., 45., 48.,  9.,
        45., 53.,  2., 51., 53., 45.,  7., 51., 51., 57.,  3., 39., 45.,
        59., 47.,  2.,  9., 45., 59., 51., 51., 54., 47., 60., 45., 47.,
        60., 50., 45.,  4., 54., 51., 45., 61., 60., 58., 55., 60., 51.,
        46., 50., 55.,  3.,  4.,  2., 55., 48.,  5.,  4., 51., 50., 45.,
         0.,  2., 61., 61., 52.,  2., 51., 47., 50., 55., 60., 53., 45.,
         4., 51., 47., 59., 45., 47.,  4., 45., 54.,  4.,  4.,  0., 37.,
        41., 41.,  7.,  7.,  7., 40.,  0., 53., 50.,  0., 40., 60., 51.,
         4., 46., 46., 45.,  4., 54., 51., 45.,  2., 47.,  4., 45.,  2.,
        47., 49., 51., 46., 46., 45., 48.,  9., 45., 56., 47.,  9., 45.,
        52.,  2., 47., 60., 57., 58., 55., 60., 46., 46., 45.,  4., 54.,
        51., 45., 47.,  3.,  4., 61., 60., 55.,  3., 54., 55., 60., 53.,
        45., 60., 47.,  2.,  2., 47.,  4., 55.,  6., 51., 45., 61., 52.,
 

In [37]:
# Split the preprocessed data into a training and testing dataset
train_count = int(round(len(y) * BASE_TRAIN_PCT))
if train_count > MAX_TRAIN_COUNT and MAX_TRAIN_COUNT > -1:
    print(f"Train count would be {train_count}")
    train_count = int(MAX_TRAIN_COUNT)
print(f"Train count is {train_count}")

test_count = len(y) - train_count
if test_count > MAX_TEST_COUNT and MAX_TEST_COUNT > -1:
    print(f"Test count would be {test_count}")
    test_count = int(MAX_TEST_COUNT)
print(f"Test count is {test_count}")

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_count, test_size=test_count, random_state=SPLIT_SEED)
print( "Initial counts: ", len(X), len(y), len(X_train), len(X_test), len(y_train), len(y_test) )

Train count would be 144960
Train count is 10000
Test count would be 183280
Test count is 10000
Initial counts:  193280 193280 10000 10000 10000 10000


In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((10000, 512), (10000, 512))

# Tensorflow Callbacks
Custom layer activation function, loss and accuracy functions, and model save checkpoint.

In [39]:
# Custom output activation, forcing output to be within range -- but not rounding it off
def modulo_output(x):
    return tf.math.mod(x, OUTPUT_MAX)

# Training checkpoint to save after each epoch, if it is a new best model:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=BEST_PATH,
    monitor="loss",
    mode="min",
    save_best_only=True,
    save_weights_only=False,
    verbose=1)

# Custom loss function, adapted from code generated by Copilot
def modulo_distance_loss(y_true, y_pred):
    """ Custom loss function to compute the modulo distance. 
    Args: 
        y_true: True values (ground truth). 
        y_pred: Predicted values. 
        modulo: The modulo value to apply -- hard coded.
    Returns: 
        The computed loss. 
    """ 
    # Compute the raw difference
    diff = tf.abs(y_true - y_pred)
    # Apply modulo operation to handle wrap-around cases
    mod_diff = tf.math.mod(diff, CUSTOM_LOSS_MODULO)
    # Ensure the distance is within the range [0, CUSTOM_LOSS_MODULO/2]
    loss = tf.minimum(mod_diff, CUSTOM_LOSS_MODULO - mod_diff) 
    return tf.reduce_mean(loss)

# Custom accuracy function, counterpart to the loss function above.
# !!! I strongly suspect this is wrong. It's never as high as it seems it should be,
#     at least when inferring keys and getting good results.
def modulo_distance_accuracy(y_true, y_pred):
    diff = tf.abs(y_true - y_pred)
    mod_diff = tf.math.mod(diff, CUSTOM_LOSS_MODULO)
    loss = tf.minimum(mod_diff, CUSTOM_LOSS_MODULO - mod_diff)

    good_part = tf.math.subtract(CUSTOM_LOSS_MODULO / 2, loss)
    accuracy = tf.math.divide(good_part, CUSTOM_LOSS_MODULO / 2)

    return tf.reduce_mean(accuracy)

# Hyperband Tuning

In [40]:
%%time

GO_FAST = True

MAX_EPOCHS_PER_MODEL = 15 # Meant to get a decent idea of parameter, not create a final model
HYPERBAND_ITERATIONS = 1  # "Number of times to iterate over the full Hyperband algorithm"
EXECUTIONS_PER_TRIAL = 1  # Training from scratch
SEARCH_FIT_EPOCHS = 7     # Epochs for each attempt to do a fit, I think. Not sure how this relates to MAX_EPOCHS_PER_MODEL.
OVERWRITE = True          # I'm hoping to be able to interrupt a run and resume it later

CHOICES_PROCESSING_UNITS = [1, CHUNK_SIZE // 4, CHUNK_SIZE, CHUNK_SIZE * 2]
CHOICES_USE_SIGMOID = [True, False]
CHOICES_USE_SCALER = [True, False] # Only relevant when using Sigmoid
CHOICES_USE_OUTPUT_LIMITER = [True, False]
CHOICES_OPTIMIZER = ["adamax", "sgd", "RMSProp"]

if GO_FAST:
    MAX_EPOCHS_PER_MODEL = 2
    SEARCH_FIT_EPOCHS = 2
    CHOICES_PROCESSING_UNITS = [1, 8]
    CHOICES_OPTIMIZER = ["adamax"]

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    processing_units = hp.Choice("Processing_Units", CHOICES_PROCESSING_UNITS)
    use_sigmoid = hp.Choice("Sigmoid", CHOICES_USE_SIGMOID)    
    use_output_limiter = hp.Choice("Output_Limiter", CHOICES_USE_OUTPUT_LIMITER)
    optimizer = hp.Choice("Optimizer", CHOICES_OPTIMIZER)

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=CHUNK_SIZE, output_dim=processing_units, name="Embedding_Input"))
    
    model.add(tf.keras.layers.LSTM(processing_units, name="LSTM"))

    if use_sigmoid:
        model.add(tf.keras.layers.Dense(units=processing_units, activation="sigmoid", name="Sigmoid"))
        use_scaler = hp.Choice("Scaler", CHOICES_USE_SCALER)
        if use_scaler:
            model.add(tf.keras.layers.Rescaling(scale=OUTPUT_MAX, offset=0, name="Rescaler")) # Input is 0-1
    
    if use_output_limiter:
        model.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation=modulo_output, name="Output_Limiter"))
    else:
        model.add(tf.keras.layers.Dense(OUTPUT_SIZE, name="Linear_Output"))

    # Compile the model
    if USE_CUSTOM_LOSS:
        loss = modulo_distance_loss
        metrics = [modulo_distance_accuracy]
    else:
        loss = LOSS_METRIC
        metrics = [MAIN_ACCURACY_METRIC]
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    return model


# Run the kerastuner search for best hyperparameters
if TUNE_NETWORK:

    if USE_CUSTOM_LOSS:
        objective = kt.Objective("modulo_distance_accuracy", direction="max")
    else:
        objective = kt.Objective(f"val_{MAIN_ACCURACY_METRIC}", direction="max")

    tuner = kt.Hyperband(
        create_model,
        objective=objective,
        max_epochs=MAX_EPOCHS_PER_MODEL,
        hyperband_iterations=HYPERBAND_ITERATIONS,
        executions_per_trial=EXECUTIONS_PER_TRIAL,
        overwrite=OVERWRITE)
    tuner.search(X_train_scaled,y_train,epochs=SEARCH_FIT_EPOCHS,validation_data=(X_test_scaled,y_test))
    
    best_hyper = tuner.get_best_hyperparameters(1)[0]
    print(f"Best Hyper Values: {best_hyper.values}")
    
    nn = tuner.get_best_models(1)[0]
    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Best Model: Loss: {model_loss}, Accuracy: {model_accuracy}")


CPU times: user 146 μs, sys: 47 μs, total: 193 μs
Wall time: 388 μs


# Model Reload /Creation

In [41]:
if LOAD_BEST_MODEL:
    print(f"Loading model from {BEST_PATH}")
    nn = tf.keras.models.load_model(BEST_PATH)
elif BUILD_NETWORK:
    print("Building new model")
    in_shape = (CHUNK_SIZE,)
    nn = tf.keras.models.Sequential()

    nn.add(tf.keras.layers.Embedding(input_dim=CHUNK_SIZE, output_dim=PROCESSING_UNITS, name="Embedding_Input"))
    nn.add(tf.keras.layers.LSTM(PROCESSING_UNITS, name="LSTM"))
    #nn.add(tf.keras.layers.Dense(units=PROCESSING_UNITS, activation="sigmoid", name="Sigmoid"))
    #nn.add(tf.keras.layers.Rescaling(scale=OUTPUT_MAX, offset=0, name="Rescaler")) # Input is 0-1
    
    if USE_CUSTOM_OUTPUT_ACTIVATION:
        nn.add(tf.keras.layers.Dense(OUTPUT_SIZE, activation=modulo_output, name="Output_Limiter"))
    else:
        nn.add(tf.keras.layers.Dense(OUTPUT_SIZE, name="Linear_Output"))
else:
    print("Nothing to do here. Hopefully you got a model somewhere above...")
        
# Check the structure of the model
nn.summary()

Building new model


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Embedding_Input (Embedding)     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Limiter (Dense)          │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Model Training

In [ ]:
if TRAIN_MODEL:
    print(f"Training model")

    if USE_CUSTOM_LOSS:
        loss = modulo_distance_loss
        metrics = [modulo_distance_accuracy]
    else:
        loss = LOSS_METRIC
        metrics = [MAIN_ACCURACY_METRIC]
    
    if SAVE_BEST_MODEL:
        callbacks = [model_checkpoint_callback]
    else:
        callbacks = None
    
    # Compile the Sequential model together and customize metrics
    nn.compile(loss=loss, optimizer=OPTIMIZER, metrics=metrics)
    
    # Fit the model to the training data
    fit_model = nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=BATCH_SIZE)

# Evaluate the model using the test data
print("Evaluating...")
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Training model
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 9.5546 - modulo_distance_accuracy: 0.6918
Epoch 1: loss improved from inf to 9.52666, saving model to ./best.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - loss: 9.5545 - modulo_distance_accuracy: 0.6918
Epoch 2/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 9.4405 - modulo_distance_accuracy: 0.6955
Epoch 2: loss improved from 9.52666 to 9.41823, saving model to ./best.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - loss: 9.4403 - modulo_distance_accuracy: 0.6955
Epoch 3/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 9.3690 - modulo_distance_accuracy: 0.6978
Epoch 3: loss improved from 9.41823 to 9.35985, saving model to ./best.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 9.3690 - modulo_distance_accuracy: 0.6978
Epoch 4/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 9.3358 - modulo_distance_accuracy: 0.6988
Epoch 4: loss improved from 9.35985 to 9.33539, saving model to ./best.ker

# Model Usefulness Spot-Check

In [ ]:
def decode_chunks_with_model(chunks: list[list], model, scaler, input_already_scaled = True) -> list[list]:
    if input_already_scaled:
        return model.predict(chunks)
    else:
        return model.predict(scaler.transform(chunks))

def decode_text_with_model(ciphertext: str, model, scaler) -> str:
    offset_chunks = helpers.chunkify(encoders.string_to_offsets(ciphertext), CHUNK_SIZE)
    decoded_chunks = decode_chunks_with_model(offset_chunks, model, scaler, input_already_scaled = False)
    rounded = np.rint(decoded_chunks.flatten()).astype(int)
    return encoders.offsets_to_string(rounded)

def infer_key_with_model(ciphertext: str, model, scaler) -> int:
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    keys = model.predict(scaler.transform(chunks))
    key = int(round(np.median(keys)))
    return key

if INFER_TEXT:    
    CHUNKS_TO_CHECK = 2
else:
    CHUNKS_TO_CHECK = 20

cipher_file_db = sid_to_c[list(sid_to_c.keys())[0]][0]
ciphertext_path = cipher_file_db.path
ciphertext = helpers.read_text_file(ciphertext_path)
ciphertext = ciphertext[0:CHUNK_SIZE * CHUNKS_TO_CHECK]
    
if INFER_TEXT:    
    print("Decoded   : ", decode_text_with_model(ciphertext, nn, X_scaler))
if INFER_KEY:
    with db.get_session() as session:
        correct_key = int(db.get_key_by_id(session, cipher_file_db.key_id).value)
    print("Correct Key: ", correct_key)
    
    inferred_key = infer_key_with_model(ciphertext, nn, X_scaler)
    print("Inferred Key: ", inferred_key)

chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
print(nn.predict(scaler.transform(chunks)))

nn.summary()